# RNN framework

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

References:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

In [1]:
from __future__ import division, print_function
import tensorflow as tf
from tensorflow.contrib import rnn


In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print ("Done......!")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Done......!


In [3]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network parameters
num_inputs = 28 # MNIST data input(image : 28 * 28)
time_steps = 28
num_hidden = 128 # hidden layer num of features
num_classes = 10

In [4]:
# define tf Graph()
X = tf.placeholder(tf.float32, shape=[None, time_steps, num_inputs])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

In [5]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [6]:
def RNN(x, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, time_steps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [7]:
logits = RNN(X, weights,biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_op = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_op, tf.float32))

# Initial the model

init = tf.global_variables_initializer()


In [8]:
# start training
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, training_steps + 1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape([batch_size, time_steps, num_inputs])
        # Run training function
        sess.run(train_op, feed_dict = {X:batch_x, Y:batch_y})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})
            
            print("Step " + str(step) + ", Minibatch Loss = "\
                 "{:.4f}".format(loss) + ", Training Accuracy = " + \
                  "{:.3f}".format(acc))
                  
    print("Optimization finished !")
                  
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape([-1, time_steps, num_inputs])
    test_labels = mnist.test.labels[:test_len]
    print("Test accuracy: ", \
         sess.run(accuracy, feed_dict={X: test_data, Y: test_labels}))

Step 1, Minibatch Loss = 2.2893, Training Accuracy = 0.234
Step 200, Minibatch Loss = 0.2029, Training Accuracy = 0.914
Step 400, Minibatch Loss = 0.2037, Training Accuracy = 0.953
Step 600, Minibatch Loss = 0.1705, Training Accuracy = 0.945
Step 800, Minibatch Loss = 0.0432, Training Accuracy = 0.992
Step 1000, Minibatch Loss = 0.0714, Training Accuracy = 0.984
Step 1200, Minibatch Loss = 0.1269, Training Accuracy = 0.961
Step 1400, Minibatch Loss = 0.0403, Training Accuracy = 0.984
Step 1600, Minibatch Loss = 0.0752, Training Accuracy = 0.977
Step 1800, Minibatch Loss = 0.0535, Training Accuracy = 0.969
Step 2000, Minibatch Loss = 0.0793, Training Accuracy = 0.961
Step 2200, Minibatch Loss = 0.0365, Training Accuracy = 0.977
Step 2400, Minibatch Loss = 0.0695, Training Accuracy = 0.977
Step 2600, Minibatch Loss = 0.0100, Training Accuracy = 1.000
Step 2800, Minibatch Loss = 0.0245, Training Accuracy = 0.992
Step 3000, Minibatch Loss = 0.0273, Training Accuracy = 0.992
Step 3200, Mini